<a href="https://colab.research.google.com/github/porterpup/flights5/blob/main/flight_two_stage_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unmount drive

In [15]:
from google.colab import drive

# Unmount Google Drive using the shell command
print("Attempting to unmount Google Drive...")
# Use the fusermount command to unmount the drive
!fusermount -uz /content/drive
print("Google Drive unmount attempt complete.")

# You can optionally check if the mount point is empty after unmounting
import os
mountpoint = '/content/drive'
# Check if the directory exists and is empty.
# os.listdir will raise FileNotFoundError if the directory doesn't exist after unmounting,
# which is a good indicator it was unmounted.
if not os.path.exists(mountpoint) or not os.listdir(mountpoint):
    print(f"Google Drive successfully unmounted from {mountpoint}.")
else:
    print(f"Warning: Google Drive mount point {mountpoint} is not empty after unmount attempt.")

Attempting to unmount Google Drive...
Google Drive unmount attempt complete.
Google Drive successfully unmounted from /content/drive.


# Mount

In [16]:
from google.colab import drive
import os
import shutil

# Check if the mountpoint exists and is not empty
mountpoint = '/content/drive'

# --- REMOVE this block as it is causing the error and is not necessary for mounting ---
# if os.path.exists(mountpoint) and os.path.isdir(mountpoint) and os.listdir(mountpoint):
#     print(f"Removing existing files from {mountpoint}...")
#     for item in os.listdir(mountpoint):
#         # Skip hidden files and directories (like .shortcut-targets-by-id)
#         if item.startswith('.'):
#             print(f"Skipping hidden item: {item}")
#             continue

#         item_path = os.path.join(mountpoint, item)
#         if os.path.isfile(item_path):
#             os.remove(item_path)
#         elif os.path.isdir(item_path):
#             shutil.rmtree(item_path)
#     print(f"Contents of {mountpoint} removed.")
# --- End of block to remove ---


# Mount Google Drive - this handles the mounting process and should not require pre-cleaning
print(f"Attempting to mount Google Drive at {mountpoint}...")
drive.mount(mountpoint)
print("Google Drive mount attempt complete.")

Attempting to mount Google Drive at /content/drive...
Mounted at /content/drive
Google Drive mount attempt complete.


# Cell 1 - Mount Drive - Only run once

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cell 2 - Installing packages

In [18]:
!pip install pandas numpy scikit-learn lightgbm airportsdata
!pip install git+https://github.com/meteostat/meteostat-python.git


  Cloning https://github.com/meteostat/meteostat-python.git to /tmp/pip-req-build-5gy7uk74
  Running command git clone --filter=blob:none --quiet https://github.com/meteostat/meteostat-python.git /tmp/pip-req-build-5gy7uk74
  Resolved https://github.com/meteostat/meteostat-python.git to commit fc9f1ba5ff9b8f9f2a60e3ba8f3a724fe538c6b3
  Preparing metadata (setup.py) ... done


# Cell 3 - Imports & Helper Functions

In [19]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

# Convert HHMM (e.g. 0530 → 330 minutes) to “minutes since midnight.”
def hhmm_to_minutes(val):
    try:
        val = int(val)
    except:
        return np.nan
    return (val // 100) * 60 + (val % 100)

# Compute ARR_DELAY = max(actual_arrival_min – scheduled_arrival_min, 0)
def compute_arr_delay(row):
    try:
        arr_min = int(row['ARR_TIME'] // 100) * 60 + int(row['ARR_TIME'] % 100)
        return max(arr_min - row['CRS_ARR_MIN'], 0)
    except:
        return np.nan


# Cell 4 - Congif & File Path

In [43]:
# Cell 4 – Write & verify cache files

import os
from pathlib import Path

# 1) Define your cache directory (adjust the path to your Drive mount if needed)
CACHE = '/content/drive/MyDrive/ColabFlightsData/cache'
Path(CACHE).mkdir(parents=True, exist_ok=True)

# 2) Define full paths
FLIGHTS_CACHE   = os.path.join(CACHE, 'flights_v3.pkl')
WEATHER_CACHE   = os.path.join(CACHE, 'weather_v3.pkl')
AUGMENTED_CACHE = os.path.join(CACHE, 'augmented_v3.pkl')

# 3) Write out your DataFrames
#    (Make sure `flights`, `weather_df`, and `augmented` are already in memory)
flights.to_pickle(FLIGHTS_CACHE)
weather_df.to_pickle(WEATHER_CACHE)
augmented.to_pickle(AUGMENTED_CACHE)

# 4) Verify & report
cache_files = {
    'Flights cache':      FLIGHTS_CACHE,
    'Weather cache':      WEATHER_CACHE,
    'Augmented cache':    AUGMENTED_CACHE
}

for label, path in cache_files.items():
    if os.path.exists(path):
        size = os.path.getsize(path)
        print(f"✅ {label} found at {path} ({size:,} bytes)")
    else:
        print(f"❌ {label} MISSING at {path}")


✅ Flights cache found at /content/drive/MyDrive/ColabFlightsData/cache/flights_v3.pkl (31,286,975 bytes)
✅ Weather cache found at /content/drive/MyDrive/ColabFlightsData/cache/weather_v3.pkl (188,580 bytes)
✅ Augmented cache found at /content/drive/MyDrive/ColabFlightsData/cache/augmented_v3.pkl (554,790,008 bytes)


# Cell 5 - Build or load flights_v3.pkl

In [44]:
# Cell 5 – Build or Load “flights_v3.pkl” (filter to ORIGIN == "ATL" and compute DEP_HOUR_BIN)

# Ensure Google Drive is mounted - you might have already done this, but good to double-check
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Use force_remount to ensure it's mounted

# Verify the data folder exists after mounting
import os
if not os.path.exists(DATA_FOLDER):
    raise FileNotFoundError(f"Data folder not found after mounting: {DATA_FOLDER}. Please check the path and your Google Drive.")

if os.path.exists(FLIGHTS_CACHE):
    flights = pd.read_pickle(FLIGHTS_CACHE)
    flights.columns = flights.columns.str.upper()
    print("Loaded cached flights_v3:", flights.shape)
else:
    df_list = []
    for path in MONTH_FILES:
        # Keep the existing file existence check, but the check above is primary
        if not os.path.exists(path):
            raise FileNotFoundError(f"Missing {path}. Please check if the file exists in your Google Drive at this location.")
        df = pd.read_csv(path)
        df.columns = df.columns.str.upper()
        df_list.append(df)
    flights = pd.concat(df_list, ignore_index=True)
    print("Combined flights shape:", flights.shape)

    # 1) Keep only UA, DL, AA
    flights = flights[flights['OP_UNIQUE_CARRIER'].isin(CARRIERS)].copy()
    print("After filtering carriers:", flights.shape)

    # 2) Filter to a single airport (ATL)
    flights = flights[flights['ORIGIN'] == AIRPORT].copy()
    print(f"After filtering to {AIRPORT} only:", flights.shape)

    # 3) Parse FL_DATE → datetime & derive DAY_OF_WEEK
    flights['FL_DATE'] = pd.to_datetime(flights['FL_DATE'], errors='coerce')
    flights = flights.dropna(subset=['FL_DATE']).copy()
    flights['DAY_OF_WEEK'] = flights['FL_DATE'].dt.dayofweek + 1

    # 4) Convert scheduled HHMM → minutes
    flights['CRS_DEP_MIN'] = flights['CRS_DEP_TIME'].apply(hhmm_to_minutes)
    flights['CRS_ARR_MIN'] = flights['CRS_ARR_TIME'].apply(hhmm_to_minutes)
    flights = flights.dropna(subset=['CRS_DEP_MIN', 'CRS_ARR_MIN']).copy()

    # 5) Only keep rows with valid departure/arrival hours (0–23)
    flights['DEP_HOUR'] = (flights['CRS_DEP_MIN'] // 60).astype(int)
    flights['ARR_HOUR'] = (flights['CRS_ARR_MIN'] // 60).astype(int)
    flights = flights[
        flights['DEP_HOUR'].between(0, 23) &
        flights['ARR_HOUR'].between(0, 23)
    ].copy()

    # 6) Create DEP_HOUR_BIN (early vs peak vs etc.)
    def hour_bin(h):
        if 4 <= h <= 7:
            return 'pre_peak'
        if 8 <= h <= 11:
            return 'morning_bank'
        if 12 <= h <= 15:
            return 'midday'
        if 16 <= h <= 19:
            return 'afternoon'
        if 20 <= h <= 23:
            return 'evening'
        return 'late_night'

    flights['DEP_HOUR_BIN'] = flights['DEP_HOUR'].apply(hour_bin)

    # 7) Numeric DISTANCE_GROUP
    flights['DISTANCE_GROUP'] = pd.to_numeric(flights['DISTANCE_GROUP'], errors='coerce')
    flights = flights.dropna(subset=['DISTANCE_GROUP']).copy()

    # 8) Ensure DEP_DELAY_NEW is numeric & drop if missing
    flights['DEP_DELAY_NEW'] = pd.to_numeric(flights.get('DEP_DELAY_NEW', np.nan), errors='coerce')
    flights = flights.dropna(subset=['DEP_DELAY_NEW']).copy()

    # 9) Save to cache
    flights.to_pickle(FLIGHTS_CACHE)
    print("Saved cached flights_v3:", flights.shape)

Mounted at /content/drive
Combined flights shape: (3461339, 24)
After filtering carriers: (1345408, 24)
After filtering to ATL only: (117540, 24)


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/ColabFlightsData/cache'

# Cell 6 Build or Load weather_v3.pkl

In [37]:
if os.path.exists(WEATHER_CACHE):
    weather_df = pd.read_pickle(WEATHER_CACHE)
    weather_df.columns = weather_df.columns.str.upper()
    print("Loaded cached weather_v3:", weather_df.shape)
else:
    from airportsdata import load
    from meteostat import Stations, Hourly

    # 1) Fetch full station inventory once
    stations = Stations().fetch()
    stations['icao'] = stations['icao'].str.upper()

    # 2) Build IATA → station_id map for US airports via ICAO = 'K' + IATA
    iata_map = load('IATA')  # e.g. {'ATL':{'lat':33.6,'lon':-84.4,...}, ...}
    station_lookup = {}
    for iata_code, info in iata_map.items():
        icao_code = f"K{iata_code.upper()}"
        match = stations.index[stations['icao'] == icao_code]
        if not match.empty:
            station_lookup[iata_code] = match[0]

    # 3) For any IATA not matched above (non-US), find nearest station in one vectorized pass
    missing_iatas = [iata for iata in iata_map if iata not in station_lookup]
    if missing_iatas:
        st_lats = stations['latitude'].values
        st_lons = stations['longitude'].values
        st_idx  = stations.index.values
        for iata_code in missing_iatas:
            info = iata_map[iata_code]
            lat0, lon0 = info['lat'], info['lon']
            dx = st_lats - lat0
            dy = st_lons - lon0
            idx_min = st_idx[(dx*dx + dy*dy).argmin()]
            station_lookup[iata_code] = idx_min

    # 4) Build flights_subset = unique (FL_DATE, ORIGIN, DEP_HOUR)
    flights_subset = flights[['FL_DATE','ORIGIN','DEP_HOUR']].drop_duplicates().reset_index(drop=True)
    flights_subset['DATE'] = flights_subset['FL_DATE'].dt.date

    # 5) Map ORIGIN → STATION via station_lookup; drop if no station
    flights_subset['STATION'] = flights_subset['ORIGIN'].map(station_lookup)
    flights_subset = flights_subset.dropna(subset=['STATION']).reset_index(drop=True)

    # 6) Add an index label so we can drop-duplicates after merge
    flights_subset = flights_subset.reset_index().rename(columns={'index':'FL_IDX'})

    weather_rows = []
    for station_id in flights_subset['STATION'].unique():
        sub = flights_subset[flights_subset['STATION'] == station_id]
        start_date = sub['FL_DATE'].min().floor('D')
        end_date   = sub['FL_DATE'].max().floor('D') + pd.Timedelta(days=1)

        hourly_data = Hourly(station_id, start_date, end_date).fetch()
        if hourly_data.empty:
            continue

        hourly_data = hourly_data.reset_index()  # “time” becomes a column
        hourly_data['DATE'] = hourly_data['time'].dt.date
        hourly_data['HOUR'] = hourly_data['time'].dt.hour

        merged = sub.merge(hourly_data, how='left', on='DATE')
        merged['HOUR_DIFF'] = (merged['HOUR'] - merged['DEP_HOUR']).abs()
        merged = merged[merged['HOUR_DIFF'] <= 1].copy()
        if merged.empty:
            continue

        merged = ( merged
                   .sort_values(['FL_IDX','HOUR_DIFF'])
                   .drop_duplicates(subset='FL_IDX', keep='first') )

        for _, row in merged.iterrows():
            if any(pd.isna(row.get(f)) for f in ['temp','wspd','prcp','pres','rhum']):
                continue
            weather_rows.append({
                'FL_DATE': pd.Timestamp(row['DATE']),
                'ORIGIN':   row['ORIGIN'],
                'DEP_HOUR': row['DEP_HOUR'],
                'TEMP_C':   row['temp'],
                'WIND_SPEED_KPH': row['wspd'],
                'PRECIP_MM': row['prcp'],
                'PRESSURE_HPA': row['pres'],
                'HUMIDITY_PCT': row['rhum']
            })

    weather_df = pd.DataFrame(weather_rows)
    weather_df.columns = weather_df.columns.str.upper()
    weather_df.to_pickle(WEATHER_CACHE)
    print("Saved cached weather_v3:", weather_df.shape)


Loaded cached weather_v3: (3222, 8)


# Cell 7 - Build or load augumented_v3.pkl

In [42]:
# Cell 7 – Full Feature Engineering & Cache (augmented_v3.pkl)

import os
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.tseries.holiday import USFederalHolidayCalendar

# 1) Paths & cache setup
CACHE = '/content/drive/MyDrive/ColabFlightsData/cache'
Path(CACHE).mkdir(parents=True, exist_ok=True)

FLIGHTS_CACHE   = os.path.join(CACHE, 'flights_v3.pkl')
WEATHER_CACHE   = os.path.join(CACHE, 'weather_v3.pkl')
AUGMENT_CACHE   = os.path.join(CACHE, 'augmented_v3.pkl')

# 2) Load base data
flights    = pd.read_pickle(FLIGHTS_CACHE)
weather_df = pd.read_pickle(WEATHER_CACHE)

# 3) Ensure FL_DATE is datetime
flights['FL_DATE'] = pd.to_datetime(flights['FL_DATE'], errors='coerce')

# 4) Helper to convert HHMM → minutes since midnight
def hhmm_to_minutes(val):
    try:
        v = int(val)
        return (v // 100) * 60 + (v % 100)
    except:
        return np.nan

# 5) Compute DEP_HOUR, CRS times, ARR_DELAY
flights['CRS_DEP_MIN'] = flights['CRS_DEP_TIME'].apply(hhmm_to_minutes)
flights['DEP_HOUR']    = (flights['CRS_DEP_MIN'] // 60).astype(int)
flights = flights[flights['DEP_HOUR'].between(0,23)].copy()

flights['CRS_ARR_MIN'] = flights['CRS_ARR_TIME'].apply(hhmm_to_minutes)
# If ARR_TIME is missing, use scheduled
flights['ARR_TIME_USE'] = flights['ARR_TIME'].fillna(flights['CRS_ARR_TIME'])
flights['ACT_ARR_MIN']  = flights['ARR_TIME_USE'].apply(hhmm_to_minutes)
flights['ARR_DELAY']    = flights['ACT_ARR_MIN'] - flights['CRS_ARR_MIN']

# 6) Merge with weather
aug = flights.merge(
    weather_df,
    how='left',
    on=['FL_DATE','ORIGIN','DEP_HOUR']
)

# 7) Drop rows missing core features
core = ['DEP_DELAY_NEW','TEMP_C','WIND_SPEED_KPH','PRECIP_MM','PRESSURE_HPA','HUMIDITY_PCT','ARR_DELAY']
aug = aug.dropna(subset=core).copy()

# 8) Rolling 3-day & 14-day avg of ARR_DELAY per carrier+origin
aug = aug.sort_values(['OP_UNIQUE_CARRIER','ORIGIN','FL_DATE'])
aug['PAST3_AVG_DELAY']  = np.nan
aug['PAST14_AVG_DELAY'] = np.nan

for (carr, orig), grp in aug.groupby(['OP_UNIQUE_CARRIER','ORIGIN']):
    grp = grp.sort_values('FL_DATE')
    s = grp.set_index('FL_DATE')['ARR_DELAY'].shift()
    roll3  = s.rolling('3d', closed='left').mean()
    roll14 = s.rolling('14d', closed='left').mean()
    aug.loc[grp.index, 'PAST3_AVG_DELAY']  = roll3.values
    aug.loc[grp.index, 'PAST14_AVG_DELAY'] = roll14.values

global_mean = aug['ARR_DELAY'].mean()
aug['PAST3_AVG_DELAY'].fillna(global_mean, inplace=True)
aug['PAST14_AVG_DELAY'].fillna(global_mean, inplace=True)

# 9) Arrival time‐of‐day bucket
aug['ARR_HOUR'] = (aug['CRS_ARR_MIN'] // 60).astype(int)
def arr_bin(h):
    if   0 <= h <= 5:   return 'late_night'
    elif 6 <= h <= 11:  return 'morning'
    elif 12 <= h <= 17: return 'afternoon'
    else:               return 'evening'
aug['ARR_DAY_PART'] = aug['ARR_HOUR'].apply(arr_bin)

# 10) US federal holiday flag
cal = USFederalHolidayCalendar()
hols = cal.holidays(start=aug['FL_DATE'].min(), end=aug['FL_DATE'].max())
aug['IS_HOLIDAY'] = aug['FL_DATE'].isin(hols).astype(int)

# 11) 24-hour rolling late rate
aug['FL_TS']   = pd.to_datetime(aug['FL_DATE'].dt.strftime('%Y-%m-%d') + ' ' +
                                aug['DEP_HOUR'].astype(str) + ':00')
aug['IS_LATE'] = (aug['ARR_DELAY'] > 0).astype(int)
pct = aug.set_index('FL_TS')['IS_LATE'].rolling('24h').mean().fillna(aug['IS_LATE'].mean())
aug['PCT_ALATLATE_24H'] = pct.values
aug.drop(columns=['FL_TS','IS_LATE'], inplace=True)

# 12) Season feature
aug['MONTH'] = aug['FL_DATE'].dt.month
def month_to_season(m):
    if m in [12,1,2]: return 'winter'
    if m in [3,4,5]:  return 'spring'
    if m in [6,7,8]:  return 'summer'
    return 'fall'
aug['SEASON'] = aug['MONTH'].apply(month_to_season).astype('category')

# 13) Severe‐weather flag
aug['IS_SEVERE_WEATHER'] = (
    (aug['PRECIP_MM'] > 2.0) &
    (aug['WIND_SPEED_KPH'] > 20.0)
).astype(int)

# 14) Hourly flight volume
vol = flights.groupby(['FL_DATE','DEP_HOUR']).size().rename('HOURLY_VOL')
aug = aug.merge(vol, how='left', on=['FL_DATE','DEP_HOUR']).fillna({'HOURLY_VOL':0})

# 15) Ensure previous‐leg delay
aug['LATE_AIRCRAFT_DELAY'] = aug['LATE_AIRCRAFT_DELAY'].fillna(0.0)

# 16) Cache the enriched dataset
aug.to_pickle(AUGMENT_CACHE)
print("🗄️  Built & cached augmented_v3:", aug.shape)

ValueError: index values must be monotonic

# Cell 8 – Stage 1 (LightGBM Binary Classifier, Categorical Columns, Early Stopping)

In [39]:
# Cell 8 – Stage 1: LightGBM binary on “> 15 min late”

import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1) Reload the enriched data
df = pd.read_pickle(AUGMENT_CACHE)
df.columns = df.columns.str.upper()

# 2) Guard any optional numeric columns
for col in ['WIND_GUST_KPH','HOURLY_VOL','PRECIP_MM']:
    if col not in df.columns:
        df[col] = 0.0

# 3) Define your feature set (including LATE_AIRCRAFT_DELAY)
features = [
    'DAY_OF_WEEK','CRS_DEP_MIN','DEP_HOUR_BIN','DEP_DELAY_NEW',
    'DISTANCE_GROUP','TEMP_C','WIND_SPEED_KPH','WIND_GUST_KPH',
    'PRECIP_MM','PRESSURE_HPA','HUMIDITY_PCT',
    'PAST3_AVG_DELAY','PAST14_AVG_DELAY','ARR_DAY_PART',
    'IS_HOLIDAY','PCT_ALATLATE_24H','SEASON','HOURLY_VOL',
    'IS_SEVERE_WEATHER','LATE_AIRCRAFT_DELAY',
    'OP_UNIQUE_CARRIER'
]

X = df[features].copy()
# 4) New target: >15 min late
y = (df['ARR_DELAY'] > 15).astype(int)

# 5) Mark categorical columns
cat_cols = ['DEP_HOUR_BIN','ARR_DAY_PART','OP_UNIQUE_CARRIER','SEASON']
for c in cat_cols:
    X[c] = X[c].astype('category')

# 6) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 7) Prepare LightGBM datasets
lgb_tr = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_cols, free_raw_data=False)
lgb_ev = lgb.Dataset(X_test,  label=y_test,  reference=lgb_tr,  categorical_feature=cat_cols, free_raw_data=False)

# 8) Train with early stopping (uses your best_params from before)
clf = lgb.train(
    best_params,
    lgb_tr,
    num_boost_round=200,
    valid_sets=[lgb_tr, lgb_ev],
    valid_names=['train','eval'],
    callbacks=[lgb.early_stopping(stopping_rounds=30)]
)

# 9) Evaluate at 0.5 cutoff
y_pred = (clf.predict(X_test, num_iteration=clf.best_iteration) > 0.5).astype(int)
print("Stage 1 (>15 min late) accuracy:", accuracy_score(y_test, y_pred))


KeyError: "['PAST3_AVG_DELAY', 'PAST14_AVG_DELAY', 'ARR_DAY_PART', 'IS_HOLIDAY', 'PCT_ALATLATE_24H', 'SEASON', 'IS_SEVERE_WEATHER'] not in index"

# Cell 9: Confusion Matrix & Classification report

In [45]:
# Cell 9 – Confusion Matrix & Classification Report

from sklearn.metrics import confusion_matrix, classification_report

# Re-compute predictions on your test set
y_pred = (clf.predict(X_test, num_iteration=clf.best_iteration) >= best_thresh).astype(int)

# 1) Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix (rows=true, cols=predicted):")
print(cm)

# 2) Detailed classification report
print("\nClassification Report:")
print(classification_report(
    y_test, y_pred,
    target_names=['≤ 15 min late (on-time)','> 15 min late']
))


Confusion Matrix (rows=true, cols=predicted):
[[18195   317]
 [  954  2628]]

Classification Report:
                         precision    recall  f1-score   support

≤ 15 min late (on-time)       0.95      0.98      0.97     18512
          > 15 min late       0.89      0.73      0.81      3582

               accuracy                           0.94     22094
              macro avg       0.92      0.86      0.89     22094
           weighted avg       0.94      0.94      0.94     22094



# Cell 10 - Stage 2: delay minutes regression on >15

In [47]:
# Cell 10 – Stage 2: delay‐minutes regression on >15 min late flights

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 1) Reload augmented to pull actual ARR_DELAY
augmented = pd.read_pickle(AUGMENT_CACHE)
augmented.columns = augmented.columns.str.upper()

# 2) Identify truly-late flights in train/test splits
late_train_idx = y_train[y_train == 1].index
late_test_idx  = y_test[y_test == 1].index

X2_train = X_train.loc[late_train_idx].copy()
y2_train = augmented.loc[late_train_idx, 'ARR_DELAY']

X2_test  = X_test.loc[ late_test_idx].copy()
y2_test  = augmented.loc[ late_test_idx, 'ARR_DELAY']

print(f"Training on {len(X2_train)} late‐flight rows; testing on {len(X2_test)}")

# 3) One‐hot encode categorical columns
cat_cols = ['DEP_HOUR_BIN','ARR_DAY_PART','SEASON','OP_UNIQUE_CARRIER']
X2_train_enc = pd.get_dummies(X2_train, columns=cat_cols, drop_first=True)
X2_test_enc  = pd.get_dummies(X2_test,  columns=cat_cols, drop_first=True)

# 4) Align train/test columns (fill any missing dummies with 0)
X2_train_enc, X2_test_enc = X2_train_enc.align(
    X2_test_enc, join='left', axis=1, fill_value=0
)

# 5) Train a RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, max_depth=8, random_state=42)
rf.fit(X2_train_enc, y2_train)

# 6) Evaluate MAE on the test subset
pred_delay = rf.predict(X2_test_enc)
mae = mean_absolute_error(y2_test, pred_delay)
print("Stage 2 MAE on truly late flights:", round(mae, 2), "minutes")


Training on 14326 late‐flight rows; testing on 3582
Stage 2 MAE on truly late flights: 16.12 minutes


# Cell 11 Prob calibration

# Threshold tuning via Precision Recall

In [40]:
# Cell 10 – Threshold tuning via Precision-Recall curve
import numpy as np
from sklearn.metrics import precision_recall_curve

# 1) Get model probabilities:
probs = clf.predict(X_test, num_iteration=clf.best_iteration)

# 2) Compute precision, recall for many thresholds:
prec, rec, thresh = precision_recall_curve(y_test, probs)

# 3) Find threshold that gives ~90% recall:
target_recall = 0.90
idx = np.argmax(rec >= target_recall)
chosen_thresh = thresh[idx]
print(f"At recall≥{target_recall:.2f}, threshold={chosen_thresh:.3f}, precision={prec[idx]:.3f}")

# 4) Re-compute confusion matrix at that new threshold:
new_pred = (probs >= chosen_thresh).astype(int)
from sklearn.metrics import confusion_matrix
print("New Confusion Matrix:", confusion_matrix(y_test, new_pred))


At recall≥0.90, threshold=0.002, precision=0.162
New Confusion Matrix: [[    0 18512]
 [    0  3582]]


# Find the F1

In [41]:
# Cell 11 – Pick threshold that maximizes F1

import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix

# 1) Get model probabilities again
probs = clf.predict(X_test, num_iteration=clf.best_iteration)

# 2) Compute precision, recall, thresholds
prec, rec, thresh = precision_recall_curve(y_test, probs)

# 3) Compute F1 scores
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-12)

# 4) Find best index
best_idx = np.nanargmax(f1_scores)
best_thresh = thresh[best_idx]
best_f1    = f1_scores[best_idx]
best_prec  = prec[best_idx]
best_rec   = rec[best_idx]

print(f"Best F1 = {best_f1:.3f} at threshold = {best_thresh:.3f}")
print(f"  Precision = {best_prec:.3f}, Recall = {best_rec:.3f}")

# 5) Show confusion matrix at that threshold
y_pred_opt = (probs >= best_thresh).astype(int)
cm_opt = confusion_matrix(y_test, y_pred_opt)
print("\nConfusion Matrix at F1-opt threshold:")
print(cm_opt)


Best F1 = 0.805 at threshold = 0.565
  Precision = 0.892, Recall = 0.734

Confusion Matrix at F1-opt threshold:
[[18195   317]
 [  953  2629]]


# Install CatBoost

In [32]:
# Cell 9 – Install CatBoost
!pip install catboost --quiet


# Ensemble of two LightGBM models (average predictions)

In [33]:
# Cell 10 – Stage 1: On-Time vs Late (CatBoostClassifier), with missing‐column guards

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1) Copy augmented and fill any missing numeric cols
df = augmented.copy()
for col in ['WIND_GUST_KPH', 'PRECIP_MM', 'PCT_ALATLATE_24H', 'HOURLY_VOL']:
    if col not in df.columns:
        df[col] = 0.0

# 2) Define features & target
features = [
    'DAY_OF_WEEK','CRS_DEP_MIN','DEP_HOUR_BIN','DEP_DELAY_NEW',
    'DISTANCE_GROUP','TEMP_C','WIND_SPEED_KPH','WIND_GUST_KPH',
    'PRECIP_MM','PRESSURE_HPA','HUMIDITY_PCT',
    'PAST3_AVG_DELAY','PAST14_AVG_DELAY','ARR_DAY_PART',
    'IS_HOLIDAY','PCT_ALATLATE_24H','SEASON','HOURLY_VOL',
    'OP_UNIQUE_CARRIER'
]
X = df[features].copy()
y = (df['ARR_DELAY'] > 0).astype(int)

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# 4) Mark categoricals
cat_features = ['DEP_HOUR_BIN','ARR_DAY_PART','SEASON','OP_UNIQUE_CARRIER']

# 5) Create Pools
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
eval_pool  = Pool(data=X_test,  label=y_test,  cat_features=cat_features)

# 6) Train CatBoost
print("\n--- Stage 1: On-Time vs Late (CatBoost) ---")
cb = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    eval_metric='Logloss',
    random_seed=42,
    early_stopping_rounds=30,
    verbose=50
)
cb.fit(train_pool, eval_set=eval_pool)

# 7) Evaluate
pred_cb = cb.predict(eval_pool)
print("CatBoost accuracy:", accuracy_score(y_test, pred_cb))



--- Stage 1: On-Time vs Late (CatBoost) ---
0:	learn: 0.6403852	test: 0.6405049	best: 0.6405049 (0)	total: 273ms	remaining: 54.3s
50:	learn: 0.4164165	test: 0.4190752	best: 0.4190752 (50)	total: 12s	remaining: 35.1s
100:	learn: 0.4052414	test: 0.4099957	best: 0.4099957 (100)	total: 17s	remaining: 16.7s
150:	learn: 0.3976820	test: 0.4053949	best: 0.4053949 (150)	total: 23.8s	remaining: 7.72s
199:	learn: 0.3918284	test: 0.4018949	best: 0.4018747 (198)	total: 28.8s	remaining: 0us

bestTest = 0.4018746848
bestIteration = 198

Shrink model to first 199 iterations.
CatBoost accuracy: 0.8287317823843577


In [ ]:
print(df.columns)